# Implementation and convergence analysis of  PLCCA (aka partially linear CCA)
Hardoon & Shawe-Taylor, 2011

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd
import cvxpy as cp
from scipy.spatial.distance import pdist, squareform
from scipy.optimize import minimize, NonlinearConstraint
import scipy
from sklearn.metrics.pairwise import pairwise_kernels, rbf_kernel
import time

In [3]:
from cca import *

In [4]:
data = pd.read_csv('ret_subset.csv')

In [5]:
ret = pd.read_csv('ret_subset.csv')

In [6]:
universe_size = 120

In [7]:
ret = np.array(ret.iloc[:252,1:universe_size])

# trading signals

quick built trading signal to obtain a kernel to test our method

In [8]:
def macd(ret, long=8, short=4, signal_span=9):
    """
    calculates the MACD momentum strategy for a single time series
    input: pandas single series
    output: numpy array of signals
    """
    short_signal = ret.ewm(span=short, adjust=False).mean()
    long_signal = ret.ewm(span=long, adjust=False).mean() 
    macd = short_signal - long_signal
    
    pos = np.zeros(len(ret))
    pos = np.where(macd>0,1,-1)
    return pos

def macd_signals(returns,long=26,short=12,signal_span=9):
    """
    function calculating all the macd signals
    input: pandas dataframe of returns
    output: pandas dataframe of signals
    """
    if type(returns) == np.ndarray:
        returns = pd.DataFrame(returns)
    signals = pd.DataFrame()
    for i in range(returns.shape[1]):
        signals['signal_{}'.format(i)] = macd(returns.iloc[:,i],long,short,signal_span)
    signals.index = returns.index
    return signals.shift(1).fillna(0)

# primal dual CCA
exact implementation

In [ ]:
s = macd_signals(ret)
tr_ret = np.array(s) * ret
kernel = gaussian_kernel(tr_ret)

In [ ]:
W,Z, output = scca_deflator(ret.T,kernel,np.arange(0,ret.shape[1],1),0,0,0.1,50,10,10)

In [ ]:
# verbose example of single portfolio
ww,ee,alphaa,betaa,muu,gammaa,corr,ress = scca(np.array(ret).T,kernel,5,1,0,0.1,50,20,20)

# Convergence analysis

In [9]:
# variables to loop over
universes = np.array([10,50,100,200,498])
sparsity = np.array([0.1,0.5,1])
overall_while = np.array([500,100,20])
convergence_w_while = np.array([100,10])
convergence_e_while = np.array([100,10])

col = ['time','assets','sparsity','overall_while','cvg_w','cvg_e','w','e','cor','res']
super_output = pd.DataFrame(columns=col)

In [ ]:
for a in universes:
    # returns
    X = np.array(data.iloc[:250,1:a])
    # kernel
    K = gaussian_kernel(X)
    for b in sparsity:
        for c in overall_while:
            for d in convergence_w_while:
                for e in convergence_e_while:
                    # get output
                    try:
                        print("currently on assets : {}, sparsity : {}, overall : {}, w cvg : {}, e cvg : {}".format(a,b,c,d,e))
                        start = time.time()
                        W,Z, output = scca_deflator(X.T,K,np.arange(0,X.shape[1],1),0,0,b,c,d,e)
                        end = time.time()
                        super_output.loc[len(super_output.index)] = [end-start,a,b,c,d,e,output['W'],output['Z'], output['cor'],output['res']]
                        #print(super_output)
                    except (RuntimeError, TypeError, NameError,ValueError,IndexError) :
                        continue

currently on assets : 10, sparsity : 0.1, overall : 500, w cvg : 100, e cvg : 100
    
currently on assets : 10, sparsity : 0.1, overall : 500, w cvg : 100, e cvg : 10
    
currently on assets : 10, sparsity : 0.1, overall : 500, w cvg : 10, e cvg : 100
    
currently on assets : 10, sparsity : 0.1, overall : 500, w cvg : 10, e cvg : 10
    
currently on assets : 10, sparsity : 0.1, overall : 100, w cvg : 100, e cvg : 100
    
currently on assets : 10, sparsity : 0.1, overall : 100, w cvg : 100, e cvg : 10
    
currently on assets : 10, sparsity : 0.1, overall : 100, w cvg : 10, e cvg : 100
    
currently on assets : 10, sparsity : 0.1, overall : 100, w cvg : 10, e cvg : 10
    
currently on assets : 10, sparsity : 0.1, overall : 20, w cvg : 100, e cvg : 100
    
currently on assets : 10, sparsity : 0.1, overall : 20, w cvg : 100, e cvg : 10
    
currently on assets : 10, sparsity : 0.1, overall : 20, w cvg : 10, e cvg : 100
    
currently on assets : 10, sparsity : 0.1, overall : 20, 

    
currently on assets : 100, sparsity : 1.0, overall : 500, w cvg : 100, e cvg : 100
    
currently on assets : 100, sparsity : 1.0, overall : 500, w cvg : 100, e cvg : 10
    
currently on assets : 100, sparsity : 1.0, overall : 500, w cvg : 10, e cvg : 100
    
currently on assets : 100, sparsity : 1.0, overall : 500, w cvg : 10, e cvg : 10
    
currently on assets : 100, sparsity : 1.0, overall : 100, w cvg : 100, e cvg : 100
    
currently on assets : 100, sparsity : 1.0, overall : 100, w cvg : 100, e cvg : 10
    
currently on assets : 100, sparsity : 1.0, overall : 100, w cvg : 10, e cvg : 100
    
currently on assets : 100, sparsity : 1.0, overall : 100, w cvg : 10, e cvg : 10
    
currently on assets : 100, sparsity : 1.0, overall : 20, w cvg : 100, e cvg : 100
    
currently on assets : 100, sparsity : 1.0, overall : 20, w cvg : 100, e cvg : 10
    
currently on assets : 100, sparsity : 1.0, overall : 20, w cvg : 10, e cvg : 100
    
currently on assets : 100, sparsity : 1.

In [ ]:
super_output